In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import time
import pandas as pd
from scipy.stats import pearsonr

In [2]:
from model.util import Normalizer
from model.database_util import get_hist_file, get_job_table_sample, collator
from model.model import QueryFormer
from model.database_util import Encoding
from model.dataset import PlanTreeDataset
from model.trainer import eval_workload, train

In [3]:
data_path = './data/imdb/'

In [4]:
class Args:
    bs = 1024
    lr = 0.001
    epochs = 200
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    device = 'cuda:0'
    newpath = './results/full/cost/'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [12]:
hist_file = get_hist_file(data_path + 'histogram_string.csv')
cost_norm = Normalizer(-3.61192, 12.290855)
card_norm = Normalizer(1,100)

In [13]:
encoding_ckpt = torch.load('checkpoints/encoding.pt')
encoding = encoding_ckpt['encoding']
checkpoint = torch.load('checkpoints/cost_model.pt', map_location='cpu')

In [14]:
from model.util import seed_everything
seed_everything()

In [15]:
model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = True, use_hist = True, \
                 pred_hid = args.pred_hid
                )

In [16]:
_ = model.to(args.device)

In [17]:
to_predict = 'cost'

In [18]:
imdb_path = './data/imdb/'
full_train_df = pd.DataFrame()
for i in range(18):
    file = imdb_path + 'plan_and_cost/train_plan_part{}.csv'.format(i)
    df = pd.read_csv(file)
    full_train_df = full_train_df.append(df)

val_df = pd.DataFrame()
for i in range(18,20):
    file = imdb_path + 'plan_and_cost/train_plan_part{}.csv'.format(i)
    df = pd.read_csv(file)
    val_df = val_df.append(df)
table_sample = get_job_table_sample(imdb_path+'train')

Loaded queries with len  100000
Loaded bitmaps


In [22]:
train_ds = PlanTreeDataset(full_train_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)
val_ds = PlanTreeDataset(val_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)

KeyboardInterrupt: 

In [32]:
crit = nn.MSELoss()
model, best_path = train(model, train_ds, val_ds, crit, cost_norm, args)

Epoch: 0  Avg Loss: 8.969336767525723e-07, Time: 25.264997482299805
Median: 1.1219273127562
Mean: 1.5001168806901493
Epoch: 20  Avg Loss: 7.359510177694675e-07, Time: 511.70180654525757
Median: 1.09213378634675
Mean: 1.4151437854169988
Epoch: 40  Avg Loss: 6.910426060332813e-07, Time: 995.7079734802246
Median: 1.084024536037849
Mean: 1.3903284630263693
Epoch: 60  Avg Loss: 6.389468571999007e-07, Time: 1508.131840467453
Median: 1.0761674608116432
Mean: 1.361751297308446


KeyboardInterrupt: 

In [29]:
methods = {
    'get_sample' : get_job_table_sample,
    'encoding': encoding,
    'cost_norm': cost_norm,
    'hist_file': hist_file,
    'model': model,
    'device': args.device,
    'bs': 512,
}

In [30]:
_ = eval_workload('job-light', methods)

Loaded queries with len  70
Loaded bitmaps
Median: 1.7423273945760955
Mean: 18.3382122430979
Corr:  0.8826572962359877


In [31]:
_ = eval_workload('synthetic', methods)

Loaded queries with len  5000
Loaded bitmaps
Median: 1.0567257417479474
Mean: 1.5938958870383626
Corr:  0.985217209834686
